In [ ]:
knitr::opts_chunk$set(eval=FALSE, results=FALSE, message=FALSE, warning=FALSE, error=FALSE) # fig.path="R/RDataWrangling/figures/"
##knitr::opts_knit$set(root.dir="R/RDataWrangling", base.dir="R/RDataWrangling")

# R Data Wrangling

**Topics**

* Loading Excel worksheets
* Iterating over files
* Writing your own functions
* Filtering with regular expressions (regex)
* Reshaping data

## Setup 

### Class Structure

* Informal --- Ask questions at any time. Really!
* Collaboration is encouraged - please spend a minute introducing yourself to your neighbors!

### Software & materials

You should have R and RStudio installed --- if not:

* Download and install R: <http://cran.r-project.org>
* Download and install RStudio: <https://www.rstudio.com/products/rstudio/download/#download>

Download materials:

* Download class materials at <https://github.com/IQSS/dss-workshops/raw/master/R/RDataWrangling.zip>
* Extract materials from the zipped directory `RDataWrangling.zip` (Right-click => Extract All on Windows, double-click on Mac) and move them to your desktop!

Start RStudio and create a new project:

* On Windows click the start button and search for RStudio. On Mac
    RStudio will be in your applications folder.
* In Rstudio go to `File -> New Project`.
* Choose `Existing Directory` and browse to the `RDataWrangling` directory.
* Choose `File -> Open File` and select the blank version of the `.Rmd` file.

### Installing & using R packages

R is a modular environment that is extended by the use of **packages**.
Packages are collections of functions or commands that are designed to
perform specific tasks (e.g., fit a type of regression model). A large 
number of contributed packages are available (> 15,000). 

Using an R package is a **two step process**:

1.  Install the package onto your computer using the
 `install.packages()` function. This only needs to
 be done the **first time** you use the package.

2.  Load the package into your R session's search path 
 using the `library()` function. This needs to be done
 **each time** you use the package.

While R's built-in packages are powerful, in recent years there has
been a big surge in well-designed *contributed packages* for R. 
In particular, a collection of R packages called 
[tidyverse](https://www.tidyverse.org/) have been 
designed specifically for data science. All packages included in 
`tidyverse` share an underlying design philosophy, grammar, and 
data structures. This philosopy is rooted in the idea of "tidy data":

![](R/Rintro/images/tidy_data.png)

We will use `tidyverse` packages throughout the 
workshop, so let's install them now:

In [ ]:
# install.packages("tidyverse")

# when you install tidyverse for the first time you will be asked
# a question in the Console - please answer by typing "no" in the Console.

library(tidyverse)

library(readxl) # installed with tidyverse, but not loaded into R session

A typical workflow for using `tidyverse` packages looks like this:

![](R/Rintro/images/tidy_workflow.png)

We can also install the `rmarkdown` package, which will allow us to
combine our text and code into a formatted document at the end of 
the workshop:

In [ ]:
# install.packages("rmarkdown")
library(rmarkdown)

The following RStudio, `tidyverse`, and `rmarkdown` cheatsheets will provide a
useful reference: <https://rstudio.com/wp-content/uploads/2019/01/Cheatsheets_2019.pdf>

### Prerequisites

This is an intermediate / advanced R course:

* Assumes intermediate knowledge of R
* Relatively fast-paced

### Learning Outcomes

* Understanding R data structures
* Iteration over data structures
* Writing functions
* Regular expressions (regex)
* Reshaping data

### Workshop Outline

**Example data**

The UK [Office for National Statistics](https://www.ons.gov.uk) provides yearly
    data on the most popular boys names going back to 1996. The data is provided
    separately for boys and girls and is stored in Excel spreadsheets.

**Overall Goal** 

Our mission is to extract and graph the **top 100** boys names in England and Wales for every year since 1996. 
 
![goal](R/RDataWrangling/images/goal.png)

**Exercise 0: Problems with the data**

There are several things that make our goal challenging. Let's take a look at the data:

1.  Locate the files named `1996boys_tcm77-254026.xlsx` and 
    `2015boysnamesfinal.xlsx` and open them separately in a 
    spreadsheet program. 

    (If you don't have a spreadsheet program installed on
    your computer you can download one from
    https://www.libreoffice.org/download/download/). 

    What issues can you identify that might make working
    with these data difficult?

    In what ways is the format different between the two files?


**Steps to accomplish this goal:**

1.  **Explore example data to highlight problems (see Exercise 0)**

2.  **Reading data from multiple Excel worksheets into R data frames**
    + list Excel file names in a character vector
    + read Excel sheetnames into a list of character vectors
    + read Excel data for "Table 1" only into a list of data frames

3.  **Clean up data within each R data frame**
    + sort and merge columns within each data frame inside the list
    + drop missing values from each data frame
    + reshape format from wide to long

4.  **Organize the data into one large data frame and store it**
    + create a year column within each data frame in the list
    + append all the data frames in the list into one large data frame

NOTE: please make sure you close the Excel files before continuing with the
workshop, otherwise you may encounter issues with file paths when reading
the data into R.

## Working with Excel worksheets

As you can see, the data is in quite a messy state. Note that this is
not a contrived example; this is exactly the way the data came to us
from the UK government website! Let's start cleaning and organizing
it. 

Each Excel file contains a worksheet with the boy names data we want.
Each file also contains additional supplemental worksheets that we are
not currently interested in. As noted above, the worksheet of interest
differs from year to year, but always has "Table 1" in the sheet name.

The first step is to get a character vector of file names.

In [ ]:
boy_file_names <- list.files("dataSets/boys", full.names = TRUE)

Now that we've told R the names of the data files, we can start working
with them. For example, the first file is

In [ ]:
boy_file_names[1]

and we can use the `excel_sheets()` function from the `readxl` package
within `tidyverse` to list the worksheet names from this file.

In [ ]:
excel_sheets(boy_file_names[1])

### Iterating over file names with `map()`

Now that we know how to retrieve the names of the worksheets in an
Excel file, we could start writing code to extract the sheet names from
each file, e.g.,

In [ ]:
excel_sheets(boy_file_names[1])

excel_sheets(boy_file_names[2])

## ...
excel_sheets(boy_file_names[20])


This is not a terrible idea for a small number of files, but it is
more convenient to let R do the iteration for us. We could use a `for loop`,
or `sapply()`, but the `map()` family of functions from the `purrr`
package within `tidyverse` gives us a more consistent alternative, 
so we'll use that.

In [ ]:
# map(object to iterate over, function that does task within each iteration)

map(boy_file_names, excel_sheets)

### Filtering strings using regular expressions

To extract the correct worksheet names we need a way to extract
strings containing "Table 1". 

Base R provides some string manipulation capabilities 
(see `?regex`, `?sub` and `?grep`), but we will use the
`stringr` package within `tidyverse` because it is more
user-friendly. `stringr` provides functions to:

1.  detect 
2.  locate
3.  extract
4.  match
5.  replace
6.  combine
7.  split   

strings. Here we want to detect the pattern "Table 1", and only
return elements with this pattern. We can do that using the
`str_subset()` function: 

1.  The first argument to `str_subset()` is character vector we want to search in. 
2.  The second argument is a *regular expression* matching the pattern we want to retain.

If you are not familiar with regular expressions (regex), 
<http://www.regexr.com/> is a good place to start. Regex is essentially
just a programmatic way of doing operations like "find" or "find and replace"
in MS Word or Excel.

Now that we know how to filter character vectors using `str_subset()` we can
identify the correct sheet in a particular Excel file. For example,

In [ ]:
# str_subset(character_vector, regex_pattern)

# nesting functions
str_subset(excel_sheets(boy_file_names[1]), pattern = "Table 1")

# piping functions
excel_sheets(boy_file_names[1]) %>% str_subset(pattern = "Table 1")

### Writing your own functions

The next step is to retrieve worksheet names and subset them.

The `map*` functions are useful when you want to apply a function to a
list or vector of inputs and obtain the return values for each input. This 
is very convenient when a function already exists that does exactly what you
want. In the examples above we mapped the `excel_sheets()` function to
the elements of a character vector containing file names. 

However, there is no function that both:

1.  Retrieves worksheet names, and 
2.  Subsets the names

So, we will have to write one. Fortunately, writing functions in R is easy. 
Functions require 3 elements:

1.  A **name**
2.  One or more **arguments**
3.  A **body** containing computations


In [ ]:
# Anatomy of a function

# function_name <- function(arg1, arg2, ....) {
#  
#   # body of function - where stuff happens #
#
#   return( results ) 
# }

myfun <- function(x) {
  x^2
}

myfun(1:10)

myfun2 <- function(x, y) {
  z <- x^2 + y
  return(z)
}

myfun(x=1:10, y=42)

In [ ]:
get_data_sheet_name <- function(file, term){
  excel_sheets(file) %>% str_subset(pattern = term)
}

# the goal is generalization 
get_data_sheet_name(boy_file_names[1], term = "Table 1")
get_data_sheet_name(boy_file_names[1], term = "Table 2")

Now we can map this new function over our vector of file names.

In [ ]:
# map(object to iterate over, 
#     function that does task within each iteration, 
#     arguments to previous function)
 
map(boy_file_names,      # list object
    get_data_sheet_name, # function
    term = "Table 1")    # argument to previous function

## Reading Excel data files

Now that we know the correct worksheet from each file, we can actually
read those data into R. We can do that using the `read_excel()` function.

We'll start by reading the data from the first file, just to check
that it works. Recall that the actual data starts on row 7, so we want
to skip the first 6 rows. We can use the `glimpse()` function from
the `dplyr` package within `tidyverse` to view the output.

In [ ]:
tmp <- read_excel(
  path = boy_file_names[1],
  sheet = get_data_sheet_name(boy_file_names[1], term = "Table 1"),
  skip = 6
)

glimpse(tmp)

Note that R has added a suffix to each column name `...1`, `...2`,
`...3`, etc. because duplicate names are not allowed, so the suffix serves
 to disambiguate. The trailing number represents the index of the column.


## Exercise 1

  1. Write a function called `read_boys_names` that takes a file name as an argument 
     and reads the worksheet containing "Table 1" from that file. Don't forget
     to skip the first 6 rows.

In [ ]:
## 

  2. Test your function by using it to read *one* of the boys names
     Excel files.

In [ ]:
## 

  3. Use the `map()` function to create a list of data frames called `boysNames`          
     from all the Excel files, using the function you wrote in step 1.

In [ ]:
## 

## Data cleanup

Now that we've read in the data, we can see that there are some
problems we need to fix. Specifically, we need to:

1. fix column names
2. get rid of blank row and the top and the notes at the bottom
3. get rid of extraneous "changes in rank" columns if they exist
4. transform the side-by-side tables layout to a single table.

In [ ]:
# Rank 1:50 --- Names / Counts are in columns 2 and 3 
# Rank 51:100 --- Names / Counts are in columns 6 and 7
glimpse(boysNames[[1]]) 

# Rank 1:50 --- Names / Counts are in columns 2 and 3 
# Rank 51:100 --- Names / Counts are in columns 7 and 8
glimpse(boysNames[[10]]) 

# Rank 1:50 --- Names / Counts are in columns 2 and 3 
# Rank 51:100 --- Names / Counts are in columns 8 and 9
glimpse(boysNames[[20]]) 

In short, we want to go from this:

![messy](R/RDataWrangling/images/messy.png)

to this:

![tidy](R/RDataWrangling/images/clean.png)

There are many ways to do this kind of data manipulation in R. We're
going to use the `dplyr` and `tidyr` packages from within `tidyverse`
to make our lives easier.

### Selecting columns

Next we want to retain just the `Name...2`, `Name...6`, `Count...3` and `Count...7` columns. 
We can do that using the `select()` function:

In [ ]:
boysNames[[1]]

boysNames[[1]] <- select(boysNames[[1]], Name...2, Name...6, Count...3, Count...7)
boysNames[[1]]

### R data types and structures

We've now encountered several different data types and data structures. Let's take a step back and survey the options available in R.

**Data structures:**

In R, the most foundational data structure is the **vector**, which comes in two basic forms:

1. **atomic**: only hold elements of the same type; they are **homogeneous**
2. **lists**: can hold elements of different types; they are **heterogeneous**

`NULL` is closely related to vectors and often serves the role of a generic zero length vector. 

![vector tree](R/RDataWrangling/images/summary_tree.png)

From these two forms, the following six structures are derived:

| Type           | Elements       | Description                                                                                                       |
|:---------------|:---------------|:------------------------------------------------------------------------------------------------------------------|
| atomic vector  | homogeneous    | contains elements of the same **type**, one of: character, integer, double, logical, or complex                   |
| array          | homogeneous    | an atomic vector with attributes giving dimensions (1, 2, or >2)                                                  |
| matrix         | homogeneous    | an array with 2 dimesions                                                                                         |
| factor         | homogeneous    | an atomic integer vector containing only predefined values, storing categorical data                              |
| list           | heterogeneous  | a container whose elements are not restricted to a single mode and can encompass any mixture of data types        |
| data.frame     | heterogeneous  | a rectangular list with elements (columns) containing atomic vectors of equal length                              |

Each vector can have **attributes**, which is a named list of arbitrary metadata that can include the vector's **dimensions** and its **class**. The latter is a property assigned to an object that determines how generic functions operate with it, and thus which **methods** are available for it. The class of an object can be queried using the `class()` function.

You can learn more details about R data structures here: <https://adv-r.hadley.nz/vectors-chap.html>

**Data types:**

There are four primary types of atomic vectors. Collectively, integer and double vectors are known as numeric vectors. You can query the **type** of an object using the `typeof()` function.

| Type                    | Description                                          |
|:------------------------|:-----------------------------------------------------|
| character               | "a", "swc"                                           |
| integer                 | 2L (the L tells R to store this as an integer)       |
| double (floating point) | 2, 15.5                                              |
| logical                 | TRUE, FALSE                                          |

![vector tree](R/RDataWrangling/images/summary_tree_atomic.png)

**Coercion:**

If heterogeneous elements are stored in an atomic vector, R will **coerce** the vector to the simplest type required to store all the information. The order of coercion is roughly: logical -> integer -> numeric -> complex -> character -> list. For example:

In [ ]:
x <- c(1, 2, 3)
typeof(x)

x <- c(1, 2, 3, "a")
typeof(x)

### List indexing

Now that we know about data structures more generally, let's focus on the list structure we created for `boysNames`. 
Why are we using **double brackets** `[[` to index this list object, instead of the single brackets `[` we used to index atomic vectors?

![list indexing](R/RDataWrangling/images/indexing_lists.png)

In [ ]:
# various data structures
numbers <- 1:10
letters <- LETTERS[1:4]
dat <- head(mtcars)
x <- 237

# combine in a list
mylist <- list(numbers, letters, dat, x)

# indexing the list
mylist[2]
class(mylist[2]) # a list

mylist[[2]]
class(mylist[[2]]) # a character vector

### Dropping missing values

Next we want to remove blank rows and rows used for notes. An easy way
to do that is to use `drop_na()` from the `tidyr` package within `tidyverse`
to remove rows with missing values.

In [ ]:
boysNames[[1]]

boysNames[[1]] <- boysNames[[1]] %>% drop_na()

boysNames[[1]]

## Exercise 2

  1. Write a function called `namecount` that takes a data frame as an 
     argument and returns a modified version, which keeps only columns that
     include the strings `Name` and `Count` in the column names.
     HINT: see the `?matches` function. 

In [ ]:
## 

  2. Test your function on the first `data.frame` in the list of boys
     names data.

In [ ]:
## 

  3. Use the `map()` function to each `data.frame` in the list of boys
     names data and save it to the list called `boysNames`.

In [ ]:
## 

### Re-arranging into a single table

Our final task is to re-arrange the data so that it is all in a single
table instead of in two side-by-side tables. For many similar tasks
the `gather()` function in the `tidyr` package is useful, but in this
case we will be better off using a combination of `select()` and
`bind_rows()`.

![](R/RDataWrangling/images/wide_vs_long.png)

In [ ]:
boysNames[[1]]

first_columns <- select(boysNames[[1]], Name = Name...2, Count = Count...3)
second_columns <- select(boysNames[[1]], Name = Name...6, Count = Count...7)

bind_rows(first_columns, second_columns)

## Exercise 3

**Cleanup all the data**

In the previous examples we learned how to drop empty rows with
`drop_na()`, select only relevant columns with `select()`, and re-arrange
our data with `select()` and `bind_rows()`. In each case we applied the
changes only to the first element of our `boysNames` list.

1.  Create a new function called `cleanupNamesData` that:

In [ ]:
# 1) subsets data to include only those columns that include the term `Name` and `Count` and apply listwise deletion

# 2) subset two separate data frames, with first and second set of `Name` and `Count` columns

# 3) append the two datasets


2.  Your task now is to use the `map()` function to apply each of these
transformations to all the elements in `boysNames`. 

NOTE: some Excel files include extra blank columns between the first and second 
set of `Name` and `Count` columns, resulting in different numeric suffixes
for the second set of columns. You will need to use a regular expression
to match each of these different column names. HINT: see the `?matches`
function.

In [ ]:
## 

## Data organization & storage

Now that we have the data cleaned up and augmented, we can turn our attention to organizing and storing the data.

### One `data.frame` for each year

Right now we have a list of data frames, one for each year. This is not a bad way to go. It has the advantage of making it easy to work with individual years; it has the disadvantage of making it more difficult to examine questions that require data from multiple years. To make the arrangement of the data clearer it helps to name each element of the list with the year it corresponds to.

In [ ]:
head(boysNames) %>% glimpse()

head(boy_file_names)

# use regex to extract years from filenames
Years <- str_extract(boy_file_names, pattern = "[0-9]{4}")
Years

names(boysNames) # returns NULL - no names in the list

# assign years to list names
names(boysNames) <- Years 

names(boysNames) # returns the years as list names

head(boysNames) %>% glimpse() 

### One big `data.frame`

While storing the data in separate data.frames by year makes some sense,
many operations will be easier if the data is simply stored in one big
data.frame. We've already seen how to turn a list of data.frames into a
single data.frame using `bind_rows()`, but there is a problem; The year
information is stored in the names of the list elements, and so
flattening the data.frames into one will result in losing the year
information! Fortunately it is not too much trouble to add the year
information to each data.frame before flattening.

In [ ]:
# apply name of the list element (.y) as a new column in the data.frame (.x)
boysNames <- imap(boysNames, ~ mutate(.x, Year = as.integer(.y)))

boysNames[1]

## Exercise 4

**Make one big data.frame**

1.  Turn the list of boys names `data.frames` into a single `data.frame`. HINT: see `?bind_rows`.

In [ ]:
## 

2.  Create a new directory called `all` within `dataSets` and write the data to a `.csv` file.
    HINT: see the `?dir.create` and `?write_csv` functions.

In [ ]:
## 

3.  What were the five most popular names in 2013?

In [ ]:
## 

4.  How has the popularity of the name "ANDREW" changed over time?

In [ ]:
## 

## Exercise solutions

### Ex 0: prototype

>     Locate the files named `1996boys_tcm77-254026.xlsx` and 
>     `2015boysnamesfinal.xlsx` and open them separately in a 
>      spreadsheet program. 
>
>     (If you don't have a spreadsheet program installed on
>     your computer you can download one from
>     https://www.libreoffice.org/download/download/). 
>
>     What issues can you identify that might make working
>     with these data difficult?
>
>     In what ways is the format different between the two files?

1.  Multiple Excel sheets in each file, each with a different name, but each file contains a `Table 1`.
2.  The data does not start on row one. Headers are on row 7, followed by a blank line, followed by the actual data.
3.  The data is stored in an inconvenient way, with ranks 1-50 in the first set of columns and ranks 51-100 in a second set of columns.
4. The second worksheet `2015boysnamesfinal.xlsx` contains extra columns between the data of interest, resulting in the second set of columns (ranks 51-100) being placed in a different position.
5.  The year from which the data comes is only reported in the Excel file name, not within the data itself.
6.  There are notes below the data.

These differences will make it more difficult to automate
re-arranging the data since we have to write code that can handle
different input formats.

### Ex 1: prototype

In [ ]:
  ## 1. Write a function that takes a file name as an argument and reads
  ##    the worksheet containing "Table 1" from that file.
 
read_boys_names <- function(file, sheet_name) {
  read_excel(
    path = file,
    sheet = get_data_sheet_name(file, term = sheet_name),
    skip = 6
  )
}
  
  ## 2. Test your function by using it to read *one* of the boys names
  ##    Excel files.

read_boys_names(boy_file_names[1], sheet_name = "Table 1") %>% glimpse()


  ## 3. Use the `map` function to read data from all the Excel files,
  ##    using the function you wrote in step 1.

boysNames <- map(boy_file_names, read_boys_names, sheet_name = "Table 1")

### Ex 2: prototype

In [ ]:
  ## 1. Write a function that takes a `data.frame` as an argument and
  ##   returns a modified version, which keeps only columns that
  ##   include the strings `Name` and `Count` in the column names.
  ##   HINT: see the `?matches` function.

  namecount <- function(data) {
      select(data, matches("Name|Count"))
  }
     
  ## 2. Test your function on the first `data.frame` in the list of boys
  ##    names data.

  namecount(boysNames[[1]])
  
  ## 3. Use the `map` function to each `data.frame` in the list of boys
  ##    names data.

  boysNames <- map(boysNames, namecount)

### Ex 3: prototype

There are different ways you can go about it. Here is one:


In [ ]:
## 1.  Your task now is to use the `map()` function to apply each of these
##     transformations to all the elements in `boysNames`. 

## NOTE: some Excel files include extra blank columns between the first and second 
## set of `Name` and `Count` columns, resulting in different numeric suffixes
## for the second set of columns. You will need to use a regular expression
## to match each of these different column names. HINT: see the `?matches` function.

cleanupNamesData <- function(file){

  # subset data to include only those columns that include the term `Name` and `Count`
  subsetted_file <- file %>%
    select(matches("Name|Count")) %>%
    drop_na()

  # subset two separate data frames, with first and second set of `Name` and `Count` columns 
  first_columns <- select(subsetted_file, Name = Name...2, Count = Count...3) 

  second_columns <- select(subsetted_file, Name = matches("Name...6|Name...7|Name...8"),
                                           Count = matches("Count...7|Count...8|Count...9"))

  # append the two datasets
  bind_rows(first_columns, second_columns)
}


## test it out on the second data.frame in the list
boysNames[[2]] %>% glimpse() # before cleanup
boysNames[[2]] %>% cleanupNamesData() %>% glimpse() # after cleanup

## apply the cleanup function to all the data.frames in the list
boysNames <- map(boysNames, cleanupNamesData)

### Ex 4: prototype

Working with the data in one big data.frame is often easier.

In [ ]:
## 1.  Turn the list of boys names `data.frames` into a single `data.frame`.

boysNames <- bind_rows(boysNames)
glimpse(boysNames)


## 2.  Create a new directory called `all` within `dataSets` and write the data to a `.csv` file. 
##     HINT: see the `?dir.create` and `?write_csv` functions.

dir.create("dataSets/all")

write_csv(boysNames, "dataSets/all/boys_names.csv")


## 3.  What were the five most popular names in 2013?   

boysNames %>% 
  filter(Year == 2013) %>%
  arrange(desc(Count)) %>%
  head()


## How has the popularity of the name "ANDREW" changed over time?

andrew <- filter(boysNames, Name == "ANDREW")

ggplot(andrew, aes(x = Year, y = Count)) +
    geom_line() +
    ggtitle("Popularity of Andrew, over time")

## Wrap-up

### Feedback

These workshops are a work in progress, please provide any feedback to: help@iq.harvard.edu

### Resources

* IQSS 
    + Workshops: <https://dss.iq.harvard.edu/workshop-materials>
    + Data Science Services: <https://dss.iq.harvard.edu/>
    + Research Computing Environment: <https://iqss.github.io/dss-rce/>

* HBS
    + Research Computing Services workshops: <https://training.rcs.hbs.org/workshops>
    + Other HBS RCS resources: <https://training.rcs.hbs.org/workshop-materials>
    + RCS consulting email: <mailto:research@hbs.edu>
    
* R
    + Learn from the best: <http://adv-r.had.co.nz/>; <http://r4ds.had.co.nz/>
    + R documentation: <http://cran.r-project.org/manuals.html>
    + Collection of R tutorials: <http://cran.r-project.org/other-docs.html>
    + R for Programmers (by Norman Matloff, UC--Davis) <http://heather.cs.ucdavis.edu/~matloff/R/RProg.pdf>
    + Calling C and Fortran from R (by Charles Geyer, UMinn) <http://www.stat.umn.edu/~charlie/rc/>
    + State of the Art in Parallel Computing with R (Schmidberger et al.) <http://www.jstatso>|.org/v31/i01/paper

